In [ ]:
from os import listdir
import numpy as np
from PIL import Image
import tensorflow as tf
start_dir = './tiny-imagenet-200/'

#List all catgories
imgCats = listdir(start_dir + 'train')
catNum = len(imgCats)
catToLabelDict = {}
imgFileNameArray = []

#One-hot encoding
for i in range(catNum):
    label = np.zeros(catNum).astype(np.float32)
    label[i] = 1.0
    catToLabelDict[imgCats[i]] = label
    for img in listdir(start_dir + 'train/' + imgCats[i] + '/images'):
        imgFileNameArray.append([imgCats[i], img]) #Put file name in array

#Shuffle data
imgFileNameArray = np.array(imgFileNameArray)
rnd_order = np.arange(len(imgFileNameArray))
np.random.shuffle(rnd_order)
imgFileNameArray = imgFileNameArray[rnd_order]


#Build batch
BATCH_SIZE = 160
x_batch = []
y_batch = []

counter = 0
x_one_batch = []
y_one_batch = []
#make batch
for i in range(len(imgFileNameArray)):
    if counter < BATCH_SIZE:
        x_one_batch.append(imgFileNameArray[i])
        #print(imgFileNameArray[i])
        y_one_batch.append(catToLabelDict[imgFileNameArray[i][0]])
        counter += 1
    else:        
        x_batch.append(x_one_batch)
        y_batch.append(y_one_batch)
        #re-init
        x_one_batch = []
        y_one_batch = []
        counter = 1
        x_one_batch.append(imgFileNameArray[i])
        y_one_batch.append(catToLabelDict[imgFileNameArray[i][0]])
#append last batch
x_batch.append(x_one_batch)
y_batch.append(y_one_batch)
#print(x_batch[0][0])

set_size = 125 #read 125 batch in memory

#actually read image
def read_image(offset):
    print('Reading images....')
    image_set = np.zeros([set_size, BATCH_SIZE, 64, 64, 3]) #64*64*3 => image size
    label_set = np.zeros([set_size, BATCH_SIZE, catNum]) #cats
    for i in range(offset * set_size, (offset + 1) * set_size):
        for batch_i in range(len(x_batch[i])):
            batch_y = y_batch[batch_i] #xy have same length
            img_dir = x_batch[i][batch_i]
            img = Image.open(start_dir + 'train/' + img_dir[0] + '/images/' + img_dir[1])
            img.load()
            arr = np.asarray(img)
            if(arr.shape == (64, 64)):
                arr = np.stack((arr,) * 3, axis = -1) #change to 3-channel if black-white
            image_set[i - offset * set_size][batch_i] = (arr - 128) / 255
            label_set[i - offset * set_size] = batch_y
    print('Finishing reading batch {0} to {1}'.format(offset * set_size, (offset + 1) * set_size - 1))
    #print(image_set[0][0])
    return image_set.astype(np.float32), label_set

def read_val_image():
    print('Reading validation images....')
    image_set = []
    label_set = []
    f = open(start_dir + 'val/val_annotations.txt')
    for pic_cat in f.readlines():
        entry = pic_cat.split()
        file_name = entry[0]
        cat = entry[1]
        img = Image.open(start_dir + 'val/images/' + file_name)
        img.load()
        arr = np.asarray(img)
        if(arr.shape == (64, 64)):
            arr = np.stack((arr,) * 3, axis = -1) #change to 3-channel if black-white
        image_set.append((arr - 128) / 255)
        label_set.append(catToLabelDict[cat])
    
    #do ???
    image_set = np.split(np.asarray(image_set).astype(np.float32), 4)
    label_set = np.split(np.asarray(label_set), 4)
    print('Finishing reading validation images.')
    return image_set, label_set

#define nn
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    #print(shape)
    #initial = tf.random_uniform(shape, minval = 0, maxval = 1, dtype=tf.float32)
    #initial = np.ones(shape).astype(np.float32) * 0.1
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    #initial = tf.random_uniform(shape, minval = 0, maxval = 1, dtype=tf.float32)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def lrelu(x):
    alpha = 0.1
    return tf.maximum(x, alpha * x)

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 64, 64, 3]) # 64x64 image
ys = tf.placeholder(tf.float32, [None, catNum])
keep_prob = tf.placeholder(tf.float32)
#x_image = tf.reshape(xs, [-1, 64, 64, 3]) #?? must edit

filter_size = 5

## conv1 layer ##
W_conv1 = weight_variable([filter_size, filter_size, 3, 16]) # patch 3*3, in size 3, out size 32
b_conv1 = bias_variable([16])
#h_conv1 = tf.nn.relu(conv2d(xs, W_conv1) + b_conv1) # output size 64x64x32
h_conv1 = lrelu(conv2d(xs, W_conv1) + b_conv1) # output size 64x64x32
h_pool1 = max_pool_2x2(h_conv1)                                         # output size 32x32x32

## conv2 layer ##
W_conv2 = weight_variable([filter_size, filter_size, 16, 32]) # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([32])
#h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 32x32x64
h_conv2 = lrelu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 32x32x64
h_pool2 = max_pool_2x2(h_conv2)                                         # output size 16x16x64

## conv3 layer ##
W_conv3 = weight_variable([filter_size, filter_size, 32, 64]) # patch 5x5, in size 64, out size 128
b_conv3 = bias_variable([64])
#h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3) # output size 16x16x128
h_conv3 = lrelu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)                                         # output size 8*8*128

## fc1 layer ##
W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool3_flat = tf.reshape(h_pool3, [-1, 8 * 8 * 64])
#h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
h_fc1 = lrelu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
#h_fc1_drop = h_fc1

## fc2 layer ##
W_fc2 = weight_variable([1024, 512])
b_fc2 = bias_variable([512])
#h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2 = lrelu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
#h_fc2_drop = h_fc2

## fc3 layer ##
W_fc3 = weight_variable([512, catNum])
b_fc3 = bias_variable([catNum])
fc3_logits = tf.matmul(h_fc2_drop, W_fc3) + b_fc3
prediction = tf.nn.softmax(fc3_logits)

# the error between prediction and real data
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))       # loss
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction)))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = ys, logits = fc3_logits))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
#train_step = tf.train.MomentumOptimizer(1e-2, 0.8).minimize(cross_entropy)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(100):
    for set_i in range(5): #等等改
        image_set, label_set = read_image(set_i)
        sum_loss = 0
        for batch_i in range(len(image_set)):
            #_, loss, dd1 = sess.run([train_step, cross_entropy, h_pool1], feed_dict={xs: image_set[batch_i], ys: label_set[batch_i], keep_prob: 0.5})
            _, loss = sess.run([train_step, cross_entropy], feed_dict={xs: image_set[batch_i], ys: label_set[batch_i], keep_prob: 0.5})
            sum_loss += loss
            #print(sess.run(tf.reduce_sum(dd1)))
        print('loss:', sum_loss / len(image_set))
        #print(image_set[0][1][20][20])
        #print('WC1:', dd1)
        #print('BC1:', dd2)
        del(image_set)
        del(label_set)
        
        #Validate process
        correct_num = 0
        image_set, label_set = read_val_image()
        for val_i in range(len(image_set)):
            y_pre = sess.run(prediction, feed_dict={xs: image_set[val_i], keep_prob: 1}) #應該是這行就錯了
            correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(label_set[val_i], 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            #result, d1, d2, d3, d4 = sess.run([accuracy, h_conv1, h_conv2, h_conv3, tf.argmax(y_pre, 1)], feed_dict={xs: image_set[val_i], ys: label_set[val_i], keep_prob: 1})
            result = sess.run(accuracy, feed_dict={xs: image_set[val_i], ys: label_set[val_i], keep_prob: 1})
            #print(y_pre[1])
            #print("val:", image_set[val_i][1][23][45])
            #print("HC1:", np.count_nonzero(d1) / d1.size)
            #print("HC2:", np.count_nonzero(d2) / d2.size)
            #print("HC3:", np.count_nonzero(d3) / d3.size)
            print('Error rate:', (1 - result) * 100, '%')
            #print('PRED:', d4)
            #print('label_set:', d2)
            
        """    
        #for test what's going on
        fake_data = np.ones((1, 64, 64 ,3)) * 128
        y_f = sess.run([prediction, h_conv1, h_conv2, h_conv3], feed_dict={xs: fake_data, keep_prob: 1})
        print("==========Fake data test==========")
        #print("PRED:", y_f[0])
        '''print("F3O:", y_f[1])
        print("F3W:", y_f[2])
        print("F3B:", y_f[3])
        print("F2W:", y_f[4])
        print("F2B:", y_f[5])
        print("F1W:", y_f[6])
        print("F1B:", y_f[7])
        print("C3W:", y_f[8])
        print("C3B:", y_f[9])
        print("C2W:", y_f[10])
        print("C2B:", y_f[11])
        print("C1W:", y_f[12])
        print("C1B:", y_f[13])'''
        print("HC1:", np.count_nonzero(y_f[1]) / y_f[1].size)
        print("HC2:", np.count_nonzero(y_f[2]) / y_f[2].size)
        print("HC3:", np.count_nonzero(y_f[3]) / y_f[3].size)
        print("==========Fake data test over==========")
        """
        
        '''
        for val_i in range(len(image_set)):
            print("IMG:", image_set[val_i][0][0][0])
            pred = sess.run(prediction, feed_dict={xs: image_set[val_i]})
            pred = pred.flatten()
            print("PRED:", pred[0])
            #print(label_set[val_i].shape)
            #print(tf.argmax(label_set[val_i]), tf.argmax(pred))
            true_label, pred_label = sess.run([tf.argmax(label_set[val_i]), tf.argmax(pred)])
            print(true_label, pred_label)
            if(true_label == pred_label):
                correct_num += 1
            #print(correct_num, end = " ")
        print('error rate:', correct_num / len(image_set))
        '''
            

In [ ]:
a = {}
a['hi'] = (19, 24)
a['yo'] = (16, "ggggg")
print(a[1])

In [ ]:
loc = './tiny-imagenet-200/val/val_annotations.txt'
f = open(loc)
for p in f.readlines():
    e = p.split()
    print(e[0], e[1])

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_xs, batch_ys = mnist.train.next_batch(100)
print(batch_xs[0])
print(batch_ys)

In [1]:
for i in range(5, 8):
    print(i)

5
6
7


In [ ]:
a = [["1","2"], ["3","4"]]
for b in a:
    print(b)

In [ ]:
def test():
    return 123, 456

a,b = 3,4
with test() as w,h:
    print(w,h)

In [ ]:
#try keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0], y_train[0])

In [ ]:
import numpy as np
a = np.ones((1,3))
print(a)
print(np.atleast_3d(a))

In [2]:
from os import listdir
import os
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np


'''
Loading data
'''
start_dir = './tiny-imagenet-200/'

#List all catgories
imgCats = listdir(start_dir + 'train')
catNum = len(imgCats)
fileLocToNum = []
catToLabelDict = {}

#One-hot encoding
for i in range(catNum):
    catToLabelDict[imgCats[i]] = i
    for img in listdir(start_dir + 'train/' + imgCats[i] + '/images'):
        loc = start_dir + 'train/' + imgCats[i] + '/images/' + img
        fileLocToNum.append((loc, i))
        
from random import shuffle 
shuffle(fileLocToNum)
#print(fileLocToNum[:10])

class training_batch:
    def __init__(self):
        pass
    
    def __read_sample(self, file_dir, label):
        img_arr = img_to_array(load_img(file_dir)) #channel first?
        if(img_arr.shape != (64, 64, 3)):
            img_arr = np.stack((img_arr, ) * 3)
        zeros = np.zeros(catNum)
        zeros[label] = 1.0
        return img_arr, zeros
    
    def __read_part_file(self):
        chunk_size = len(fileLocToNum) // 5
        print(chunk_size)
        for i in range(5):
            x_chunk, y_chunk = [], []
            for img_file in fileLocToNum[chunk_size * i:chunk_size * (i + 1)]:
                x_sample, y_sample = self.__read_sample(img_file[0], img_file[1])
                x_chunk.append(x_sample)
                y_chunk.append(y_sample)
            yield np.asarray(x_chunk), np.asarray(y_chunk)
            
    def __check_if_binary_built(self):
        status = True
        file_list = ['train.data.1.npy', 'train.data.2.npy', 'train.data.3.npy', 
                      'train.data.4.npy', 'train.data.5.npy', 'train.label.1.npy',
                      'train.label.2.npy', 'train.label.3.npy', 'train.label.4.npy',
                      'train.label.5.npy']
        for file_name in file_list:
            status = status and os.path.exists(start_dir + file_name)
        return status
    
    def __build_binary(self):
        i = 1
        print('build_start')
        for chunk_x, chunk_y in self.__read_part_file():
            np.save(start_dir + 'train.data.' + str(i) + '.npy', chunk_x)
            np.save(start_dir + 'train.label.' + str(i) + '.npy', chunk_x)
            i += 1
        print('build_finish')
    
    def get_batch(self):
        if(self.__check_if_binary_built()):
            while 1:
                for i in range(5):
                    chunk_x = np.load(start_dir + 'train.data.' + str(i + 1) + '.npy')
                    chunk_y = np.load(start_dir + 'train.label.' + str(i + 1) + '.npy')
                    counter  = 0
                    chunk_size = chunk_x.shape[0]
                    print(chunk_x.shape)
                    while (counter + 1) * batch_size < chunk_size:
                        start = counter * batch_size
                        end = (counter + 1) * batch_size
                        print(chunk_x[start:end].shape)
                        yield chunk_x[start:end], chunk_y[start:end]
                        counter += 1
                    start = counter * batch_size
                    yield chunk_x[start:chunk_size], chunk_y[start:chunk_size]
        else:
            self.__build_binary()
            self.get_batch()

class val_batch:
    def __init__(self):
        pass
    
    def __read_sample(self, file_dir, label):
        img_arr = img_to_array(load_img(file_dir)) #channel first?
        if(img_arr.shape != (64, 64, 3)):
            img_arr = np.stack((img_arr, ) * 3)
        zeros = np.zeros(catNum)
        zeros[label] = 1.0
        return img_arr, zeros
    
    def __read_part_file(self):
        f = open(start_dir + 'val/val_annotations.txt')
        chunk_size = len(f.readlines())
        x_chunk, y_chunk = [], []
        for line in f.readlines():
            lines = line.split()
            file_dir = start_dir + 'val/images/' + lines[0]
            label = catToLabelDict[lines[1]]
            x_sample, y_sample = self.__read_sample(file_dir, label)
            x_chunk.append(x_sample)
            y_chunk.append(y_sample)
        return np.asarray(x_chunk), np.asarray(y_chunk)
            
    def __check_if_binary_built(self):
        status = True
        file_list = ['val.data.1.npy', 'val.label.1.npy']
        for file_name in file_list:
            status = status and os.path.exists(start_dir + file_name)
        return status
    
    def __build_binary(self):
        i = 1
        print('val_build_start')
        for chunk_x, chunk_y in self.__read_part_file():
            np.save(start_dir + 'val.data.' + str(i) + '.npy', chunk_x)
            np.save(start_dir + 'val.label.' + str(i) + '.npy', chunk_x)
            i += 1
        print('val_build_finish')
    
    def get_batch(self):
        if(self.__check_if_binary_built()):
            while 1:
                chunk_x = np.load(start_dir + 'val.data.1.npy')
                chunk_y = np.load(start_dir + 'val.label.1.npy')
                counter  = 0
                chunk_size = chunk_x.shape[0]
                print(chunk_x.shape)
                while (counter + 1) * batch_size < chunk_size:
                    start = counter * batch_size
                    end = (counter + 1) * batch_size
                    yield chunk_x[start:end], chunk_y[start:end]
                    counter += 1
                start = counter * batch_size
                yield chunk_x[start:chunk_size], chunk_y[start:chunk_size]
        else:
            self.__build_binary()
            self.get_batch()

batch_size = 256
epochs = 100
input_shape = (64, 64, 3)

model = Sequential()

model.add(Conv2D(64,(3, 3),padding='same',activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=3,padding='same',activation='relu'))
#model.add(Conv2D(128, kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
                 
model.add(Conv2D(256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(256,kernel_size=3,padding='same',activation='relu'))
#model.add(Conv2D(256,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
                 
model.add(Conv2D(128,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(128,kernel_size=3,padding='same',activation='relu'))
#model.add(Conv2D(128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))   
                

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(catNum, activation='softmax'))


optimizer = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
              metrics = ['accuracy'])

def get_training_batch():
    b = training_batch()
    return b.get_batch()

def get_val_batch():
    b = val_batch()
    return b.get_batch()

model.fit_generator(get_training_batch(),
                    steps_per_epoch = len(fileLocToNum) // batch_size + 1,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = get_val_batch(),
                    validation_steps = 10000 // batch_size + 1)

# Save model and weights
save_dir = './'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, 'test')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

score = model.evaluate(x_test, y_test, verbose=0) #沒test set 會爆
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


Epoch 1/100
(20000, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)
(256, 64, 64, 3)


ValueError: Error when checking target: expected dense_3 to have 2 dimensions, but got array with shape (256, 64, 64, 3)

In [6]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os

batch_size = 128
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
391/391 [==============================] - 14s 36ms/step - loss: 2.2754 - acc: 0.1270 - val_loss: 2.1270 - val_acc: 0.1817
Epoch 2/100
391/391 [==============================] - 12s 32ms/step - loss: 2.0214 - acc: 0.2105 - val_loss: 1.8066 - val_acc: 0.3088
Epoch 3/100
391/391 [==============================] - 12s 31ms/step - loss: 1.8194 - acc: 0.2934 - val_loss: 1.6539 - val_acc: 0.3611
Epoch 4/100
391/391 [==============================] - 12s 32ms/step - loss: 1.6821 - acc: 0.3609 - val_loss: 1.5100 - val_acc: 0.4473
Epoch 5/100
391/391 [==============================] - 12s 32ms/step - loss: 1.5476 - acc: 0.4267 - val_loss: 1.3288 - val_acc: 0.5154
Epoch 6/100
391/391 [==============================] - 12s 31ms/step - loss: 1.4354 - acc: 0.4753 - val_loss: 1.2481 - val_acc: 0.5530
Epoch 7/100
391/391 [==============================] - 12s 32ms/step - loss: 1.349

In [ ]:
import numpy as np
np.empty([0, 64, 64, 3])